In [24]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
import os
import shap

In [25]:
targetName = list(range(0,7))
modelloML = ['lr','dt','rf','mlp']
fileCSV = ['datasetArtisticBackgroundFINAL.csv', 'datasetScientificBackgroundFINAL.csv']

In [26]:
model_reg = ['lr',
                'dt',
                'rf',
                'mlp']

param_lr = [{'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}]

param_dt = [{'max_depth': [5, 10, 15, 20, 25, 30]}]

param_mlp = [{'hidden_layer_sizes': [(5,), (10,), (15,)],
             'activation': ['tanh', 'relu'],
             'solver': ['sgd', 'adam'],
             'alpha': [0.0001, 0.05],
             'learning_rate': ['constant','adaptive'],}]

param_rf = [{'bootstrap': [True, False],
             'max_depth': [10, 20, 30],
             'max_features': ['auto', 'sqrt'],
             'min_samples_leaf': [1, 2, 4],
             'min_samples_split': [2],}]

models_regression = {
    'lr': {'name': 'Linear Regression',
           'estimator': LinearRegression(),
           'param': param_lr,
          },
    'dt': {'name': 'Decision Tree',
        'estimator': DecisionTreeRegressor(random_state=42),
        'param': param_dt,
          },
    'rf': {'name': 'Random Forest',
           'estimator': RandomForestRegressor(random_state=42),
           'param': param_rf,
          },
    'mlp': {'name': 'Multi Linear Perceptron',
            'estimator': MLPRegressor(random_state=42),
            'param': param_mlp
           },
}

In [27]:
music_categories ={'Rock':0,'Classic':1,'Pop':2,'R&B':3,'Country':4,'Electronic':5,'Jazz':6,'Folk':7,'Indie':8,'Metal':9,'Rap':10}
    
AItechnique_categories ={'searchStartegy':0,'planning':1,'constraints':2}
    

In [31]:
for file in fileCSV:
    dataset = pd.read_csv(file, sep=';')
    X = dataset[['timeDuration', 'nMovements', 'movementsDifficulty', 'AItechnique', 'robotSpeech',    'acrobaticMovements', 'movementsRepetition', 'musicGenre', 'movementsTransitionsDuration', 'humanMovements', 'balance', 'speed', 'bodyPartsCombination', 'musicBPM', 'sameStartEndPositionPlace', 'headMovement', 'armsMovement', 'handsMovement', 'legsMovement', 'feetMovement']]
    X.loc[:,'musicGenre']=X.musicGenre.apply(lambda x: music_categories[x])
    X.loc[:,'AItechnique']=X.AItechnique.apply(lambda x: AItechnique_categories[x])
    index_target= dataset.iloc[:,-7:]    
    back = file.split('dataset')[1].split('Background')[0]
    
    for t in targetName:
        list_ind_t = index_target.columns.values.tolist()
        targetN = list_ind_t[t]
        y = dataset[targetN]
        
        for m in modelloML:
            mod_grid = GridSearchCV(models_regression[m]['estimator'], models_regression[m]['param'], cv=5, return_train_score = False, scoring='neg_mean_squared_error', n_jobs = 8)
            data_train, data_test, target_train, target_test = train_test_split(
                                                                    X, y, test_size=0.2, random_state=42)
            model = mod_grid            
            model.fit(data_train, target_train)
            
            X100 = shap.utils.sample(X, 100)
            explainer = shap.Explainer(model.predict,X)
            shap_values = explainer(X)

            shap.plots.beeswarm(shap_values, show=False)
            plt.savefig('Results-%s/Results-%s/%s/Plot/beeswarm.jpg' %(back, m,targetN), bbox_inches='tight')
            plt.clf()
            plt.cla()
            plt.close()
            
            print("Saved beeswarm",file,targetN,m )
            
            shap.plots.bar(shap_values, show=False)
            plt.savefig('Results-%s/Results-%s/%s/Plot/bar.jpg' %(back, m,targetN), bbox_inches='tight')
            plt.clf()
            plt.cla()
            plt.close()
            
            print("Saved bar",file,targetN,m )
            
            shap.plots.heatmap(shap_values, show=False)
            plt.savefig('Results-%s/Results-%s/%s/Plot/heatmap.jpg' %(back, m,targetN), bbox_inches='tight')
            plt.clf()
            plt.cla()
            plt.close()
            
            print("Saved heatmap",file,targetN,m )
            
            
            


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


Permutation explainer: 404it [00:13,  7.94it/s]                                                                        


Saved beeswarm datasetArtisticBackgroundFINAL.csv EvaluationChoreographyStoryTelling lr
Saved bar datasetArtisticBackgroundFINAL.csv EvaluationChoreographyStoryTelling lr
Saved heatmap datasetArtisticBackgroundFINAL.csv EvaluationChoreographyStoryTelling lr


Permutation explainer: 404it [00:13,  7.07it/s]                                                                        


Saved beeswarm datasetArtisticBackgroundFINAL.csv EvaluationChoreographyStoryTelling dt
Saved bar datasetArtisticBackgroundFINAL.csv EvaluationChoreographyStoryTelling dt
Saved heatmap datasetArtisticBackgroundFINAL.csv EvaluationChoreographyStoryTelling dt


Permutation explainer: 404it [01:53,  3.25it/s]                                                                        


...

Saved beeswarm datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility dt
Saved bar datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility dt
Saved heatmap datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility dt


Permutation explainer: 404it [01:52,  3.28it/s]                                                                        


Saved beeswarm datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility rf
Saved bar datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility rf
Saved heatmap datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility rf


Permutation explainer: 404it [00:21,  9.18it/s]                                                                        


Saved beeswarm datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility mlp
Saved bar datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility mlp
Saved heatmap datasetScientificBackgroundFINAL.csv EvaluationHumanReproducibility mlp
